# --------------------------------------------------------------------------------------

# Algoritmo - Determinação das velocidades GPS
### _Autor: Antônio Claudio Dutra Batista_
### _Orientador: Francisco Moraes de Oliveira Neto_
### _Metodologia: Velocidades médias entre os registros do GPS_
''' As informações dos veículos no trecho foram obtidos pela propria base do gps pelo, dado que o fornecimento dessas informações nor ultimos anos ja vem com dados das linhas que cada veiculo da frota fazer parte naquele momento da operação, assim é possivel saber os veculos das linhas de interesse.'''

# 1.0  Leitura das Bases

In [ ]:
# Importando bibliotecas e funções necessárias 
import folium 
import numpy as np 
import pandas as pd
import geopandas as gpd
from shapely import Point
from shapely.geometry import Point, Polygon
import plotly.graph_objs as go
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
# !pip install geopy
from geopy.distance import geodesic 

## 1.1.0 leitura do base - GPS (Tratamento INICIAL)

In [ ]:
# Lista para armazenar os dataframes
grupos_dias = []

# Loop para ler os arquivos CSV e concatená-los 
for i in range(1, 31):  # Início, fim (não inclusivo)
    # Lendo do arquivo CSV
    df = pd.read_csv(f'arquivo_paitt_diario_2023-11-{str(i).zfill(2)}.csv', sep=',')
    
    # Selecionando as primeiras 4 colunas e a última coluna
    df = df.iloc[:, [1, 2, 3, 5, -1]]
    
    # Renomeando as colunas pela posição
    df.columns = ['latitude', 'longitude', 'momento', 'linha', 'id_veiculo']
    
    # Armazenando o dataframe na lista
    grupos_dias.append(df)

In [ ]:
# Juntando em um mesmo df
GPS_I_H =  pd.concat(grupos_dias) 

In [ ]:
GPS_I_H

In [ ]:
# Informações da base
GPS_I_H.info()

In [ ]:
# Formatando coluna com informaçãoes de data
GPS_I_H['momento'] = pd.to_datetime(GPS_I_H['momento'],format= '%Y%m%d%H%M%S')

In [ ]:
GPS_I_H.info()

In [ ]:
GPS_I_H.sort_values("momento", inplace = True)

In [ ]:
GPS_I_H

## 1.1.1 lnspeção inicial

In [ ]:
GPS_I_H['linha'].unique()

In [ ]:
GPS_I_H.isnull().sum()

In [1]:
''' NENHUM VALOR NULO DOS REGISTROS DE 11_2023! '''

' NENHUM VALOR NULO DOS REGISTROS DE 11_2023! '

In [2]:
''' VERIFICANDO REGISTROS ENTRE 6-7HORAS DA MANHÃ NO DIA 01 e 06 e 07 (dias uteis do mes) na região de interesse: '''

' VERIFICANDO REGISTROS ENTRE 6-7HORAS DA MANHÃ NO DIA 01 e 06 e 07 (dias uteis do mes) na região de interesse: '

In [ ]:
# FILTRANDO DADOS DO VECIULO EM TRECHO DE INTERESSE
registro_regiao = GPS_I_H[(GPS_I_H.latitude>=-3.73667)
    & (GPS_I_H.longitude>=-38.56944) & (GPS_I_H.longitude<=-38.54167)
                                & (GPS_I_H.latitude<=-3.73083)]

In [ ]:
# FILTRO DIA 01
dia_01_registros = registro_regiao[(registro_regiao.momento>=
                '2023-11-01 06:00:00') & (registro_regiao.momento<='2023-11-01 07:00:00')]

In [ ]:
dia_01_registros

In [ ]:
dia_01_registros['id_veiculo'].unique()

In [3]:
''' OU SEJA SOMENTE REGISTRO DE 3 VEIUCLO NO DIA 01 NO TRECHO DE INTERESSE DE 6-7hs! '''

' OU SEJA SOMENTE REGISTRO DE 3 VEIUCLO NO DIA 01 NO TRECHO DE INTERESSE DE 6-7hs! '

In [ ]:
# FILTRO DIA 06
dia_06_registros = registro_regiao[(registro_regiao.momento>=
                '2023-11-06 06:00:00') & (registro_regiao.momento<='2023-11-06 07:00:00')]

In [ ]:
dia_06_registros

In [ ]:
dia_06_registros['id_veiculo'].unique()

In [4]:
''' MESMA SITUAÇÃO, POUCOS REGISTROS DE VECIULOS NESSE HORARIO! '''

' MESMA SITUAÇÃO, POUCOS REGISTROS DE VECIULOS NESSE HORARIO! '

In [ ]:
# FILTRO DIA 07
dia_07_registros = registro_regiao[(registro_regiao.momento>=
                '2023-11-07 06:00:00') & (registro_regiao.momento<='2023-11-07 07:00:00')]

In [ ]:
dia_07_registros

In [5]:
''' O MESMO ACONTECE NO DIA 07, OU SEJA, NESSE HORARIO DE 6-7hs POSSIVELMENTE O GPS NÃO ATUALIZA O LAT LON CORRETAMENTE! '''

' O MESMO ACONTECE NO DIA 07, OU SEJA, NESSE HORARIO DE 6-7hs POSSIVELMENTE O GPS NÃO ATUALIZA O LAT LON CORRETAMENTE! '

# 2.0 Separando dias para geração e analises dos perfis veiculares

In [ ]:
# Criando coluna de dias da semana de 0 = segunda e 6 = domingo
GPS_I_H['dia_semana'] = GPS_I_H['momento'].dt.dayofweek

In [ ]:
# Reatribuindo dados do gps a variavel GPS_I_H para dias de segunda a sexta
filtro_semana = GPS_I_H[GPS_I_H['dia_semana'] < 5]

In [ ]:
del GPS_I_H # Excluindo explicitamente base GPS completo

In [ ]:
# Removendo dia 2,3 e 15 (feriados)
filtro_uteis = filtro_semana[((filtro_semana.momento< '2023-11-01 23:59:59') | (filtro_semana.momento> '2023-11-03 23:59:59'))
                             & ((filtro_semana.momento< '2023-11-14 23:59:59') | (filtro_semana.momento> '2023-11-15 23:59:59'))]

In [ ]:
del filtro_semana # Excluindo explicitamente base GPS com feriados

In [6]:
''' o metodo será aplocado para extração das velcoidade filtrado intervalos dos dias, assim será criada uma coluna de momento curto '''

' o metodo será aplocado para extração das velcoidade filtrado intervalos dos dias, assim será criada uma coluna de momento curto '

In [ ]:
#  Criando coluna com dados de horario, minuto e segundos de registros do GPS 
filtro_uteis['momento_curto'] = filtro_uteis['momento'].dt.strftime('%H:%M:%S')

In [ ]:
filtro_uteis.info()

In [ ]:
filtro_uteis

In [7]:
''' BASE COM COLUNA DE MOMENTO CURTO CRIADA PARA FILTRAGEM DE INTERVALOS DE INTERESSE E DEIXADO NA BASE SOMENTE OS DIAS A SEREM GERADOS E ANALISADOS OS PERFIS VEICULARES '''

' BASE COM COLUNA DE MOMENTO CURTO CRIADA PARA FILTRAGEM DE INTERVALOS DE INTERESSE E DEIXADO NA BASE SOMENTE OS DIAS A SEREM GERADOS E ANALISADOS OS PERFIS VEICULARES '

# 3.0 Extraindo velocidades veiculares a partir do GPS
''' Esse algoritimo é aplicado por dia que se deseja fazer a analise '''

## 3.1 Identificando linhas operando no trecho de interesse
''' OBS: É NECESSÁRIO DEFINIR O HORARIO PARA GERAÇÃO E ANALISES DOS PERFIS '''

In [8]:
''' SERÁ IDENTIFICADAS AS LINHAS NESSA REGIÃO, POIS NÃO TEM NUMHUMA RUA OU AVENIDA QUE CORTA O TRECHO DE INTERESSE NA TRANSVERSAL E ASSIM POSSIVELMENTE NÃO TEM OUTRA LINHAS NESSES SENTIDOS: -3.734498870509772, -38.55723488359994; -3.733084073193523, -38.55346122624892 '''

' SERÁ IDENTIFICADAS AS LINHAS NESSA REGIÃO, POIS NÃO TEM NUMHUMA RUA OU AVENIDA QUE CORTA O TRECHO DE INTERESSE NA TRANSVERSAL E ASSIM POSSIVELMENTE NÃO TEM OUTRA LINHAS NESSES SENTIDOS: -3.734498870509772, -38.55723488359994; -3.733084073193523, -38.55346122624892 '

In [ ]:
# FILTRANDO VALIDAÇÕES EM TRECHO ESPECIFICO 
validacoes_trecho = filtro_uteis[(filtro_uteis.latitude>=-3.734498870509772)
    & (filtro_uteis.longitude>=-38.55723488359994) & (filtro_uteis.longitude<=-38.55346122624892)
                                & (filtro_uteis.latitude<=-3.733084073193523)]

In [ ]:
# VALIDAÇOES NOS DIAS UTEIS DO MES NA REGIÃO DE INTERESSE 
validacoes_trecho

In [9]:
''' OBS: É INTERESSANTE FILTAR O HORARIO PARA GERAÇÃO DE PERFIS EM INTERVALOS CURTOS, PARA QUE NÃO SE TENHA SOBREPOSIÇÃO DE REGISTROS DE UM MESMO VEICULO NO MESMO HORARIO '''

' OBS: É INTERESSANTE FILTAR O HORARIO PARA GERAÇÃO DE PERFIS EM INTERVALOS CURTOS, PARA QUE NÃO SE TENHA SOBREPOSIÇÃO DE REGISTROS DE UM MESMO VEICULO NO MESMO HORARIO '

In [ ]:
# LINHAS NA REGIÃO E NO HORARIO  
linhas = validacoes_trecho['linha'].unique()
linhas

## 3.2 Identificação de veiculos na região de interesse

In [ ]:
# FILTRANDO DADOS DO VECIULO EM TRECHO DE INTERESSE (completo!)
viculos_especificos__ = filtro_uteis[(filtro_uteis.latitude>=-3.73667)
    & (filtro_uteis.longitude>=-38.56944) & (filtro_uteis.longitude<=-38.54167)
                                & (filtro_uteis.latitude<=-3.73083)]

In [10]:
''' Foi atribuido um trecho para captação dos registro do GPS antes e depois do trecho real de interesse: Comeco ((-3.73667, -38.56944)) e Fim (-3.73083, -38.54167) ''' 

' Foi atribuido um trecho para captação dos registro do GPS antes e depois do trecho real de interesse: Comeco ((-3.73667, -38.56944)) e Fim (-3.73083, -38.54167) '

In [ ]:
# Filtrando os registros no horario de pico do dia a ser extraido as velocidades
viculos_especificos_TRECHO__ = viculos_especificos__[(viculos_especificos__.momento_curto>=
                '17:00:00') & (viculos_especificos__.momento_curto<='18:00:00')]

In [ ]:
# Colocando coluna de momento em ordem
viculos_especificos_TRECHO_= viculos_especificos_TRECHO__.sort_values('momento')

In [ ]:
viculos_especificos_TRECHO_

In [ ]:
# CASO DESEJE APLICAR PARA PERFIS DE LINHA ESPECIFICA:
# viculos_especificos_TRECHO_ = viculos_especificos_TRECHO_[viculos_especificos_TRECHO_.linha== 222] # ex: linha 222!

In [ ]:
# Criando coluna geometrica
viculos_especificos_TRECHO_['geometry'] = viculos_especificos_TRECHO_.apply(lambda x: Point((float(x.latitude), float(x.longitude))), axis=1)

In [ ]:
# Criando geodataframe 
viculos_especificos_TRECHO_ = gpd.GeoDataFrame(viculos_especificos_TRECHO_, geometry='geometry')

In [ ]:
viculos_especificos_TRECHO_

## 3.3 Analise Espacial 
'' OBS: QUANDO SE CLICA NOS PONTOS NO MAPA É POSSIVEL VER OS REGISTROS E QUAIS VEICULOS ESTÃO RELAMENTE NA DIREÇÃO DO TRECHO DE ESTUDO, ASSIM SERÃO REMOVIDOS AQUELES QUE NÃO OBEDECEREM ESSA CONDIÇÃO '''

### 3.3.1 Vizualização dos dados no trecho

In [ ]:
# Criando media para visulização do mapa proximo a região dos dados geometricos
lon = viculos_especificos_TRECHO_['longitude'].mean()
lat = viculos_especificos_TRECHO_['latitude'].mean()

In [ ]:
# Criando um mapa centrado em nos registros em FORTALEZA
# mapa_viculo_especifico_TRECHO = folium.Map(location=[lat, lon], zoom_start=5)

# Função para gerar o conteúdo do popup
# def popup_content(id_veiculo, id_veiculo_clicado):
#     cor_popup = 'blue' if id_veiculo != id_veiculo_clicado else 'red'
#    return f'<div style="color: {cor_popup};">{id_veiculo}</div>'

# Criando o MarkerCluster
#marker_cluster = MarkerCluster().add_to(mapa_viculo_especifico_TRECHO)

# Adicionando os marcadores ao cluster em vez de diretamente ao mapa
#for index, row in viculos_especificos_TRECHO_.iterrows():
#    folium.Marker(
#        location=[row['latitude'], row['longitude']],
#        icon=folium.Icon(icon='glyphicon-pushpin', color='blue'),
#        popup=folium.Popup(popup_content(row['id_veiculo'], ''), max_width=300)
#    ).add_to(marker_cluster)

In [ ]:
# mapa_viculo_especifico_TRECHO

In [11]:
''' A PARTIR DESSA ANALISE É POSSIVEL VERIFICAR QUE APENAS COM O FILTRO ESPACIAL EXISTEM VEICULOS QUE NÃO ESTÃO NA ROTA DE ESTUDO '''

' A PARTIR DESSA ANALISE É POSSIVEL VERIFICAR QUE APENAS COM O FILTRO ESPACIAL EXISTEM VEICULOS QUE NÃO ESTÃO NA ROTA DE ESTUDO '

### 3.3.2 Tratamento espacial
''' Como são muitos pontos fora da rota, iremos criar um poligono e filtrar os registros dentro deste '''

In [ ]:
# Definindo o polígono da rota da Avenida Bezerra de Menezes
polygon_coords = [
    (-3.73667, -38.56944),
    (-3.736849494989972, -38.5694049332511),
    (-3.7365163965678407, -38.56766822912734),
    (-3.7353274286377487, -38.56200455534079),
    (-3.7339173396045293, -38.554907846078386),
    (-3.732780710996037, -38.54946365551903),
    (-3.7320059683399855, -38.54575810102139),
    (-3.731064497340683, -38.54163534630501),
    (-3.7315660320364357, -38.54574199080628),
    (-3.7324918066930017, -38.55026979714917),
    (-3.7335601222379684, -38.55554285351419),
    (-3.734628031910939, -38.560965217226546),
    (-3.7358612998322682, -38.56677042320314),
    (-3.7365111312489647, -38.56946314519871),
    (-3.73083, -38.54167),
    (-3.73667, -38.56944)  # Fechando o polígono com o ponto inicial
]

polygon = Polygon(polygon_coords)

In [ ]:
polygon # poligono bezerra

In [ ]:
# Verificar quais pontos estão dentro do polígono
viculos_especificos_TRECHO = viculos_especificos_TRECHO_[viculos_especificos_TRECHO_.geometry.within(polygon)]

In [ ]:
# REGISTROS DO GPS DENTRO DO POLIGONO
viculos_especificos_TRECHO

In [12]:
''' VISUALIZAÇÃO DE REGISTROS APOS FILTRO: '''

' VISUALIZAÇÃO DE REGISTROS APOS FILTRO: '

In [ ]:
lat_2 = viculos_especificos_TRECHO['latitude'].mean()
lon_2 = viculos_especificos_TRECHO['longitude'].mean()

# Criando um mapa centrado em FORTALEZA
import folium
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[lat_2, lon_2], zoom_start=5)

# Adicionando marcadores circulares ao mapa
marker_cluster = MarkerCluster().add_to(mapa)

def popup_content(id_veiculo, id_veiculo_clicado):
    cor_popup = 'blue' if id_veiculo != id_veiculo_clicado else 'red'
    return f'<div style="color: {cor_popup};">{id_veiculo}</div>'

for index, row in viculos_especificos_TRECHO.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
                        radius=10,
                        color='black',
                        fill=True,
                        fill_color='red',
                        fill_opacity=0.6,
                        popup=folium.Popup(popup_content(row['id_veiculo'], ''), max_width=300)).add_to(marker_cluster)

mapa

In [13]:
''' ASSIM, TEMOS TODOS OS REGISTROS AO LONGO DO TRECHO DE INTERESSE, MES E NO HORARIO '''

' ASSIM, TEMOS TODOS OS REGISTROS AO LONGO DO TRECHO DE INTERESSE, MES E NO HORARIO '

## _3.4 Criando lista com um df para cada veiculo das linhas filtradas_

In [14]:
''' Criando função para pegar os dados de um mesmo veiculo e no mesmo dia e separar em diferentes dfs, alem de quando a diferença no tempo for maior que 2 minutos separar em df diferente, pois possivelmente o veiculo estará no meso trecho, mas no sentido contrário. '''

' Criando função para pegar os dados de um mesmo veiculo e no mesmo dia e separar em diferentes dfs, alem de quando a diferença no tempo for maior que 2 minutos separar em df diferente, pois possivelmente o veiculo estará no meso trecho, mas no sentido contrário. '

In [ ]:
# Função para separar pelo criterio
def agrupar_por_ids(viculos_especificos_TRECHO):    
    # Agrupando dados por id_veiculo e data (extraindo apenas a data sem a hora)
    grupos = viculos_especificos_TRECHO.groupby(['id_veiculo', viculos_especificos_TRECHO['momento'].dt.date])
    
    # Lista para guardar os GeoDataFrames
    geodataframes = []
    
    # Iterando sobre os grupos 
    for _, grupo in grupos:
        # Ordenando o grupo pela coluna de data
        grupo = grupo.sort_values(by='momento')
        
        # Lista para guardar os GeoDataFrames de um mesmo dia
        daily_geodataframes = []
        temp_list = [grupo.iloc[0]]  # Lista temporária para armazenar registros
        
        # Iterando sobre o grupo ordenado
        for i in range(1, len(grupo)):
            # Verificando a diferença de tempo entre registros consecutivos
            diff = grupo.iloc[i]['momento'] - grupo.iloc[i-1]['momento']
            if diff > pd.Timedelta(minutes=2):
                # Se a diferença for maior que 2 minutos, criar um novo GeoDataFrame
                temp_df = pd.DataFrame(temp_list)
                geo_df = gpd.GeoDataFrame(temp_df, geometry=gpd.points_from_xy(temp_df.longitude, temp_df.latitude))
                daily_geodataframes.append(geo_df)
                temp_list = [grupo.iloc[i]]
            else:
                # Se a diferença for menor ou igual a 2 minutos, adicionar à lista temporária
                temp_list.append(grupo.iloc[i])
        
        # Adicionando o último grupo de registros
        if temp_list:
            temp_df = pd.DataFrame(temp_list)
            geo_df = gpd.GeoDataFrame(temp_df, geometry=gpd.points_from_xy(temp_df.longitude, temp_df.latitude))
            daily_geodataframes.append(geo_df)
        
        # Adicionando os GeoDataFrames diários à lista principal
        geodataframes.extend(daily_geodataframes)
        
    return geodataframes

In [ ]:
# Aplicando a função
viculos_especificos_TRECHO__n = agrupar_por_ids(viculos_especificos_TRECHO)

In [ ]:
# Quantidade de df com dados de mesmo veiculo em tempo seguindo e no mesmo dia
len(viculos_especificos_TRECHO__n)

In [ ]:
# Visualizando df especifico 
viculos_especificos_TRECHO__n[0]

## 3.5 Extraindo velocidades instantâneas para veiculos especificos

In [ ]:
# Colocando coluna de momento de cada df das lista
# Função para processar um DataFrame
def ordem_df(df):
    # Ordenar o DataFrame pela coluna 'momento'
    df = df.sort_values('momento')
    
    # Restaurando o índice do DataFrame após a ordenação
    df = df.reset_index(drop=True)    
    return df

# Aplicando a função para cada DataFrame na lista
viculos_especificos_TRECHO__ordem = [ordem_df(df) for df in viculos_especificos_TRECHO__n]

In [ ]:
# Criando função para criar coluna de gemetrica anterior em cada DataFrame
def processar_dataframe_dist(df):
    df['geometry_anterior'] = df['geometry'].shift(1)
    return df

In [ ]:
# Aplicando a função para cada DataFrame na lista
viculos_especificos_TRECHO_geo_ant = [processar_dataframe_dist(df) for df in viculos_especificos_TRECHO__ordem]

In [ ]:
# Acesso ao resultado, por exemplo, para o primeiro DataFrame na lista processada
primeiro_dataframe_processado = viculos_especificos_TRECHO_geo_ant[0]
primeiro_dataframe_processado

In [ ]:
# Criando função para criar coluna de momento anterior em cada DataFrame
def processar_dataframe_momento(df):
    df['momento_anterior'] = df['momento'].shift(1)
    return df

# Aplicando a função para cada DataFrame na lista
viculos_especificos_TRECHO_geo_mom_ant = [processar_dataframe_momento(df) for df in viculos_especificos_TRECHO_geo_ant]

In [ ]:
# Acessando a resultado, por exemplo, para o primeiro DataFrame na lista processada
primeiro_dataframe_processado = viculos_especificos_TRECHO_geo_mom_ant[1]
primeiro_dataframe_processado

In [ ]:
# Função para calcular distância entre pontos
def calcular_distancia(row):
    if row['geometry'] is None or row['geometry_anterior'] is None:
        return None
    return geodesic((row.geometry.x, row.geometry.y), (row.geometry_anterior.x, row.geometry_anterior.y)).meters

# Função para processar um DataFrame
def processar_distancia(df):
    df['distancia'] = df.apply(calcular_distancia, axis=1)
    return df

In [ ]:
# Aplicando a função para cada DataFrame na lista
viculos_especificos_TRECHO_dist_ = [processar_distancia(df) for df in viculos_especificos_TRECHO_geo_mom_ant]

In [ ]:
viculos_especificos_TRECHO_dist_[0]

In [ ]:
# Função para calcular a diferença de tempo
def calcular_diferenca_tempo(row):
    if pd.isnull(row['momento']) or pd.isnull(row['momento_anterior']):
        return None
    return (row['momento'] - row['momento_anterior']).total_seconds()

# Função para processar um GeoDataFrame
def processar_geodataframe_temp(gdf):
    # Aplicar a função calcular_diferenca_tempo à coluna 'momento' e criar nova coluna 'deltaT'
    gdf['deltaT'] = gdf.apply(calcular_diferenca_tempo, axis=1)
    
    # Converter a coluna 'deltaT' para o tipo de dados float
    gdf['deltaT'] = gdf['deltaT'].astype('float')
    
    # Adicionar outras operações ou transformações conforme necessário
    
    return gdf

In [ ]:
# Aplicando função 
viculos_especificos_TRECHO_dist_temp = [processar_geodataframe_temp(gdf) for gdf in viculos_especificos_TRECHO_dist_]

In [ ]:
# Função para calcular a velocidade instantânea
def calcular_velocidade_instantanea(row):
    if pd.isnull(row['distancia']) or pd.isnull(row['deltaT']) or row['deltaT'] == 0:
        return None
    return row['distancia'] / row['deltaT']

# Função para processar um GeoDataFrame
def processar_geodataframe_velocidade(gdf):
    # Calcular a velocidade instantânea
    gdf['velocidade'] = gdf.apply(calcular_velocidade_instantanea, axis=1)
    
    return gdf

In [ ]:
#########################_________Aplicando a função para cada GeoDataFrame na lista processada__________#######################
viculos_especificos_TRECHO_velocidade_tudo = [processar_geodataframe_velocidade(gdf) for gdf in 
                                              viculos_especificos_TRECHO_dist_temp]
################################################################################################################################

In [ ]:
viculos_especificos_TRECHO_velocidade_tudo[0]

In [ ]:
# Dropando colunas desnecessárias 
# Lista de colunas a serem removidas
colunas_desnecessarias = ['geometry_anterior', 'momento_anterior', 'distancia']

# Função para remover colunas desnecessárias
def remover_colunas_desnecessarias(gdf):
    # Remover as colunas desnecessárias
    gdf = gdf.drop(columns=colunas_desnecessarias, errors='ignore')
    
    # Adicionar outras operações ou transformações conforme necessário
    
    return gdf

In [ ]:
# Aplicando a função para cada GeoDataFrame na lista de velocidades
viculos_especificos_TRECHO_velocidades = [remover_colunas_desnecessarias(gdf) for gdf in viculos_especificos_TRECHO_velocidade_tudo]

In [ ]:
# Versão final da lista com df para todos os veiculos da lista (filtrando df2, que é o elemento 1 da lista!)
viculos_especificos_TRECHO_velocidades[1]

## 3.6 Adicioando dados da primeira parada 
''' Utilizando como ponto de referencia a parada inicial para perfis de velocidades de veiculos no sentido da primeira estação centro, e como referencia a ultima estação no sentido contrario '''

In [ ]:
# Trazendo base com as paradas de interesse
paradas_interesse = pd.read_excel('paradas_interesse_tudo_com_distancias.xlsx')

In [ ]:
paradas_interesse

In [ ]:
# Tramanho do trecho
display(paradas_interesse['distancia'].sum())

In [ ]:
# CRIANDO DF COM OS DADOS GEOMETRICOS DA 1ª PARADA E DA ULTIMA PARA DISTANCIA EM RELAÇÃO A UMA OU OUTRA
# USA-SE OS DADOS GEOMETRICOS DA ULTIMA PARADA QUANDO QUEREMOS OS VEICULOS NA VOLTA! 
data_ = ['2018-11-01 00:00:00']
data = pd.to_datetime(data_)
df_paradadas_  = {'latitude': [-3.736013],
                  'longitude': [-38.566300],
                  'id_veiculo': [np.nan],
                  'momento': [np.nan],
                  'geometry': [np.nan],
                  'deltaT': [np.nan],
                  'velocidade':  [np.nan],
                
        }

df_paradadas__ = pd.DataFrame(df_paradadas_)
df_paradadas__['momento'] = data
df_paradadas__

In [ ]:
# Info dos dados 
df_paradadas__.info()

In [ ]:
# Criando a informação da geometria no df
df_paradadas__['geometry'] = df_paradadas__.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [ ]:
df_paradadas__

In [ ]:
 # Convertendo a coluna 'nova_geometria' para a coluna de geometria principal
df_paradada_inicio = gpd.GeoDataFrame(df_paradadas__, geometry='geometry')

In [ ]:
df_paradada_inicio

In [ ]:
''' CRIANDO MESMO DF COM INFROMAÇÕES GEOMETRICAS DA ULTIMA PARADA '''
df_paradada_fim = df_paradada_inicio.copy()
df_paradada_fim['latitude'] = [-3.731264]
df_paradada_fim['longitude'] = [-38.543328]

 # Atualizando a coluna 'nova_geometria' para a coluna de geometria principal
df_paradada_fim['geometry'] = df_paradada_fim.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [ ]:
df_paradada_inicio

In [ ]:
df_paradada_fim

## 3.7 Filtrando dados no seguimento para cada df da lista

In [ ]:
# FILTRANDO DADOS PARA SEGUIMENTO ENTRE A PRIMERA E ULTIMA PARADA DE INTERESSE
def filtrar_por_seguimento_(gdf):
    # Aplicar o filtro
    filtro = (gdf['longitude'] >= -38.566300) & (gdf['longitude'] <= -38.543328)
    gdf_filtrado = gdf[filtro]
    
    return gdf_filtrado

# Aplicando a função para cada GeoDataFrame na lista sem colunas desnecessárias
resultados = [filtrar_por_seguimento_(gdf) for gdf in viculos_especificos_TRECHO_velocidades]

In [ ]:
# Acessando resultado para o segundo dataframe que é o elemento com indice 01
resultados[0]

## 3.7.1 Adequando colunas para padronização das distancias dos perfis
*  SE DEVE ADICIONAR OS DADOS DA ESTAÇÃO PARA PEGAR OS VEICULOS EM OPERAÇÃO NO SENTIDO A PARTIR DELA;
* EXEMPLO: VEICULOS VOLTA SE DEVE ADICOANR DADSOS DA PARADA FIM!

In [ ]:
#  ADICIONANDO DADOS DA ESTAÇÃO PARA CALCULOS DE DISTANCIAS 
from pandas import concat  

# Concatenando os dataframes, adicionando a nova coluna
for i, gpf in enumerate(resultados):
    resultados[i]  = gpf.append(df_paradada_fim, ignore_index=True)

In [ ]:
# COLOCANDO DADOS DA COLUNA MOMENOTO EM ORDEM 
for i, gpf in enumerate(resultados):
    resultados[i] = gpf.sort_values(by='momento')

In [ ]:
resultados[0]

## 3.7.2 Realizando medição das distancia em relação a parada de interesse

In [ ]:
# Crindo coluna de geometry anterior com função de calculo de distancia ja criada
resultados_ordem_dist = [processar_dataframe_dist(df) for df in resultados]

In [ ]:
resultados_ordem_dist[0]

In [ ]:
# Aplicando função de distancia também feita anteriormente
resultados_dist_tudo = [processar_distancia(df) for df in resultados_ordem_dist]

In [ ]:
resultados_dist_tudo[4]

In [ ]:
# Criando coluna com distancias acumuladas (Em relação a parada de interesse)
def calcular_distancia_acumulada(resultados, coluna_distancia):
    resultados_atualizados = []
    # Iterando sobre cada GeoDataFrame na lista 'resultados'
    for gdf in resultados_dist_tudo:
        # Calculando a distância acumulada
        gdf['distancia_acumulada'] = gdf[coluna_distancia].cumsum()

        # Adicionando o GeoDataFrame atualizado à lista
        resultados_atualizados.append(gdf)

    return resultados_atualizados

In [ ]:
# Chamando a função para calcular a distância acumulada
resultados = calcular_distancia_acumulada(resultados_dist_tudo, 'distancia')

In [ ]:
# LISTA COM DATAFRAMES COM DADOS DE IDA E VOLTA 
len(resultados)

## _3.7.3 Informações dos df´s gerados com velocidades_

In [ ]:
# INFORMAÇÕES DE CADA DF DA LISTA COM VELOCIDADES
# for dfs in resultados:
  #   display(dfs.info())

In [ ]:
# EXIBINDO CADA DF DA LISTA RESULTADOS COM VELOCIDADES
# for dfs in resultados:
  #   display(pd.DataFrame(dfs))

# 4.0 Identificação perfis no mesmo sentido
    ''' INICIALMENTE ADICIONAMOS NOS DADOS DAS VELOCIDADES A LOCALIZAÇÃO DAS ESTAÇÕES DE INTERESSE PARA MEDIR AS DISTANCIAS DOS REGISTOS EM RELAÇÃO A ELA, ENTÃO BASTA FILTRAR AQUELES PERFIS QUE OBEDECEM A LOGICA DA DISTANCIA ACUMULADA SER PEQUENA E IR CRESCENDO COM O PASSAR DO TEMPO, ESSE SENDO UM INDICATIVO DE QUE ESSE VEICULO ESTÁ NO SENTIDO DE INTERESSE ''' 

In [ ]:
# Comprimento do Trecho em analise
comprimento_trecho = 2583.53007784341
print(comprimento_trecho)

In [ ]:
# FILTRANDO SOMENTE UMA VIAGEM SEGUIDA NO SEU INTEVALO (TRECHO) DE PASSAGEM:
lista_filtrada = []

for df in resultados:
    # Filtrando as linhas onde a coluna 'distancia_acumulada' é menor que tamnho do trecho
    df_filtrado = df.query('distancia_acumulada < 2583.53007784341')
    lista_filtrada.append(df_filtrado)

In [15]:
    ''' AGORA IREMOS FILTRAR DESSA LISTA AQUELES PERFIS QUE TIVEREM MAIS DE 5 LINHAS, OU SEJA, SOMENTE OS QUE RELAMENTE ESTIVEREM NO SENTIDO TERÃO MAIS REGISTROS E OQUE NÃO TIVEREM VÃO TER POUCAS LINHAS APOS A APLICAÇÃO DO FILTRO ACIMA!  '''

' AGORA IREMOS FILTRAR DESSA LISTA AQUELES PERFIS QUE TIVEREM MAIS DE 5 LINHAS, OU SEJA, SOMENTE OS QUE RELAMENTE ESTIVEREM NO SENTIDO TERÃO MAIS REGISTROS E OQUE NÃO TIVEREM VÃO TER POUCAS LINHAS APOS A APLICAÇÃO DO FILTRO ACIMA!  '

In [ ]:
# Filtrando quantidade minima de registros no trecho 
lista_final = []

for df in lista_filtrada:
    if len(df) > 8:
        lista_final.append(df)

In [ ]:
len(lista_final) 

In [ ]:
# FILTRANDO OS PERFIS QUE A DISTANCIA ENTRE O PRIMEIRO REGISTO NO TRECHO E A PRIMERIA PARDA SEJA MENOR QUE 500 METROS
geodataframes_filtrados_dist = []

# Iterar sobre cada GeoDataFrame na lista
for geodataframe in lista_final:
    # Verificar se a primeira linha da coluna "distancia" tem um valor menor que 500 metros
    if geodataframe["distancia"][0] < 500:
        # Se sim, adicionar o GeoDataFrame à lista de GeoDataFrames filtrados
        geodataframes_filtrados_dist.append(geodataframe)

In [ ]:
geodataframes_filtrados_dist[0]

In [ ]:
# FILTRANDO PERFIS DE NO MAXIMO 100 KM/HORA (ADIMITINDO QUE SUPERIORES SERÃO DEVIDO ERROS NO GPS!)
geodataframes_filtrados_dist_vel = [gdf for gdf in geodataframes_filtrados_dist if (gdf['velocidade'] <= 27.78).all()]

In [ ]:
len(geodataframes_filtrados_dist_vel)

In [ ]:
    ####### ESSSA É A QUANTIDADE DE PERFIS NO SENTIDO DE ANALISE E HORARIO #######

In [16]:
''' Ida quando se compara a distancia com a primeira parada '''

' Ida quando se compara a distancia com a primeira parada '

In [17]:
''' Volta quando os filtrados são os que possuem distancia em relação a ultima estação (ultima sentido centro) '''

' Volta quando os filtrados são os que possuem distancia em relação a ultima estação (ultima sentido centro) '

# 5.0 Base com perfis discretizados e analises iniciais
    ''' OBS: A PARTIR DESSE TOPICO SERÃO FEITAS AS ANALISES!  ''' 

''' Assumindo que a componente de velocidade a cada trecho discretizado seja proporcional a velocidade no trecho completo '''

In [ ]:
# Discretizando a cada 5 metros
# Lista para armazenar os DataFrames resultantes
lista_velocidades_discretizadas = []

for df in geodataframes_filtrados_dist_vel: 
    # Inicializando a distância acumulada
    distancia_acumulada = 0
    
    # Obtendo o tamanho total do trecho
    tamanho_total_trecho = df['distancia_acumulada'].max()
    
    # Obtendo a data e horário do primeiro registro
    momento_atual = df.iloc[0]['momento']
    data_hora_formatada = momento_atual.strftime("%Y-%m-%d_%H-%M-%S")
    
    # Criando um DataFrame para armazenar as velocidades discretizadas
    coluna_nome = f'velocidade_{df.iloc[0]["linha"]}_{df.iloc[0]["id_veiculo"]}_{data_hora_formatada}'
    velocidades_discretizadas = pd.DataFrame(columns=['distancia', coluna_nome])
    
    linha_atual = df.iloc[0]['linha']  # Obtendo a informação da linha do primeiro DataFrame
    id_veiculo_atual = df.iloc[0]['id_veiculo']  # Obtendo a informação do id_veiculo do primeiro DataFrame
    
    # Iterando sobre os trechos do DataFrame
    for index, row in df.iterrows():
        distancia_trecho_atual = row['distancia']
        velocidade_trecho_atual = row['velocidade']
    
        # Adicionando as distâncias discretizadas ao DataFrame de velocidades discretizadas
        for distancia in range(0, int(distancia_trecho_atual) + 1, 5):
            if distancia_acumulada > tamanho_total_trecho:
                break
            velocidades_discretizadas = velocidades_discretizadas.append({'distancia': distancia_acumulada, 
                                                                          coluna_nome: velocidade_trecho_atual}, 
                                                                         ignore_index=True)
            distancia_acumulada += 5
    
    lista_velocidades_discretizadas.append(velocidades_discretizadas)

In [ ]:
lista_velocidades_discretizadas[0]

In [ ]:
# Fator de conversão de m/s para km/h
fator_conversao = 3.6

for df in lista_velocidades_discretizadas:
    # Iterando sobre as colunas do DataFrame
    for coluna in df.columns:
        # Verificando se o nome da coluna contém a palavra "velocidade"
        if 'velocidade' in coluna:
            # Convertendo a coluna de velocidade de m/s para km/h
            df[coluna] *= fator_conversao

## 5.1 Visualização e resumo dos perfis no trecho e horario especifico

In [ ]:
# Lista para armazenar os traces de velocidade de cada DataFrame
traces = []

# Iterando sobre cada DataFrame da lista
for idx, df in enumerate(lista_velocidades_discretizadas):
    # Extraindo informações da coluna de velocidade
    info_velocidade = df.columns[-1].split('_')
    linha_atual = info_velocidade[1]  # Obtendo o número da linha
    id_veiculo_atual = info_velocidade[2]  # Obtendo o ID do veículo
    
    # Criando um trace para a velocidade de cada DataFrame
    trace = go.Scatter(
        x=df['distancia'],
        y=df.iloc[:, -1],  # Utilizando a última coluna do DataFrame como velocidade
        mode='lines',
        name=f'Velocidade - Linha {linha_atual} - ID {id_veiculo_atual}',  # Nomeando o trace com informações dinâmicas
        hoverinfo='x+y+name',  # Informações mostradas ao passar o mouse sobre o gráfico
    )
    traces.append(trace)

# Preenchendo o valor nulo na primeira linha da coluna "distancia_acumulada" com zero
paradas_interesse['distancia_acumulada_paradas'].iloc[0] = 0

# Criando um trace para as paradas de interesse
trace_paradas = go.Scatter(
    x=paradas_interesse['distancia_acumulada_paradas'],  # Coordenadas x
    y=[0] * len(paradas_interesse),  # Coordenadas y (todos são 0)
    mode='markers',  # Modo de plotagem de marcadores
    marker=dict(
        symbol='triangle-up',  # Formato do marcador (triângulo para cima)
        size=10,  # Tamanho do marcador
        color='black',  # Cor do marcador
    ),
    hovertext=paradas_interesse['stop_name'],  # Texto de dica ao passar o mouse sobre os marcadores
    hoverinfo='text',  # Informações mostradas ao passar o mouse
    name='Estações BRT'  # Nome do trace
)

# Adicionando o trace das paradas de interesse à lista de traces
traces.append(trace_paradas)

# Criando o layout do gráfico
layout = go.Layout(
    title='Velocidades (veículos das linhas no trecho e horário de interesse)',
    xaxis=dict(title='Distância (m)'),
    yaxis=dict(title='Velocidade (km/h)'),
    hovermode='closest'
)

# Criando a figura com os traces e o layout
fig = go.Figure(data=traces, layout=layout)

# Exibindo o gráfico
fig.show()

In [18]:
''' SEGUE DESCRIÇÃO ESTATISTICA DE CADA PERFIL: (descomantar os codigos para visualização!) '''

' SEGUE DESCRIÇÃO ESTATISTICA DE CADA PERFIL: (descomantar os codigos para visualização!) '

In [ ]:
# for idx, df in enumerate(lista_velocidades_discretizadas):
#     # Obtendo o nome da coluna de velocidade
#     nome_coluna_velocidade = df.columns[-1]
    
    # Calculando o resumo estatístico apenas para a coluna de velocidade
#     resumo_velocidade = df[nome_coluna_velocidade].describe()
    
    # Imprimindo o resumo estatístico apenas para a coluna de velocidade
#     print(f"Resumo Estatístico do perfil de velocidade ({nome_coluna_velocidade}):")
#     print(resumo_velocidade)
#     print()  # Adiciona uma linha em branco entre os resumos estatísticos

In [19]:
''' OBSERVAR A MÉDIA DE VELOCIDADES E DISPERSÃO DOS VALORES'''

' OBSERVAR A MÉDIA DE VELOCIDADES E DISPERSÃO DOS VALORES'

## 5.2 Exportação
''' COM DADOS DE PERFIS VEICULARES DE DIFERENTES LINHAS NO MESMO INTERVALO, BASTA EXPORTALOS E GERAR PARA DEMAIS HORARIOS DE INTERSSE, DIAS E PROCEGUIR COM AS DEMAIS ANALISES! ''' 

In [ ]:
# Definindo o nome do arquivo Excel
nome_arquivo_excel = "dados_perfis_velocidades_17_18hs_volta.xlsx"

# Criando um DataFrame vazio para armazenar os dados finais
df_final = pd.DataFrame(columns=['distancia'])

# Iterando sobre cada DataFrame na lista
for idx, df in enumerate(lista_velocidades_discretizadas):
    # Adicionando a coluna de distância ao DataFrame final
    df_final['distancia'] = df['distancia']
    
    # Obtendo o nome da coluna de velocidade
    nome_coluna_velocidade = df.columns[-1]
    
    # Adicionando as colunas de velocidade discretizada ao DataFrame final
    df_final[nome_coluna_velocidade] = df.iloc[:, -1]

# Exportando o DataFrame final para o arquivo Excel
df_final.to_excel(nome_arquivo_excel, index=False)

# --------------------------------------------------------------------------------------